# Mlflow tracking

In [3]:
#
# train_diabetes.py
#
#   MLflow model using ElasticNet (sklearn) and Plots ElasticNet Descent Paths
#
#   Uses the sklearn Diabetes dataset to predict diabetes progression using ElasticNet
#       The predicted "progression" column is a quantitative measure of disease progression one year after baseline
#       http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html
#   Combines the above with the Lasso Coordinate Descent Path Plot
#       http://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html
#       Original author: Alexandre Gramfort <alexandre.gramfort@inria.fr>; License: BSD 3 clause
#
#  Usage:
#    python train_diabetes.py 0.01 0.01
#    python train_diabetes.py 0.01 0.75
#    python train_diabetes.py 0.01 1.0
#

In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets

# Load Diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [6]:
# Create pandas DataFrame for sklearn ElasticNet linear_model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = diabetes.feature_names + ['progression']
data = pd.DataFrame(d, columns=cols)

In [7]:
# Import mlflow
import mlflow
import mlflow.sklearn
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [18]:
import mlflow

In [8]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "progression" which is a quantitative measure of disease progression one year after baseline
train_x = train.drop(["progression"], axis=1)
test_x = test.drop(["progression"], axis=1)
train_y = train[["progression"]]
test_y = test[["progression"]]

In [34]:
from sklearn.model_selection import RandomizedSearchCV
alpha_list=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]
la_ratio_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
hyperparameters = dict(alpha=alpha_list,l1_ratio=la_ratio_list)
clf=RandomizedSearchCV(ElasticNet(random_state=42), hyperparameters, cv=10, verbose=0,scoring='r2')
clf.fit(train_x, train_y)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'alpha': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09], 'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='r2', verbose=0)

In [35]:
from datetime import datetime
log_results(clf, 'Test', 'ElasticNet')

Traceback (most recent call last):
  File "C:\Users\LENOVO\Anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\LENOVO\Anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\LENOVO\Anaconda3\lib\site-packages\mlflow\utils\file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file 'D:\\Data_Science\mlflow\mlruns\cv_results_matrix\meta.yaml' does not exist.


Logging parameters
Logging metrics
Logging model
Logging CV results matrix
Logging extra data related to the experiment
file:///D:/Data_Science/mlflow/mlruns/1/86cc73bfbaa24732af1483c8cc9afb5d/artifacts
runID: 544aa7a9fb8f4910bf6f1fa28cd1ebfb
Logging parameters
Logging metrics
Logging model
Logging CV results matrix
Logging extra data related to the experiment
file:///D:/Data_Science/mlflow/mlruns/1/747e8baf610b40afb905b3ae1b0f8ac3/artifacts
runID: 77ff6020632f421cb28c9affa1317da9
Logging parameters
Logging metrics
Logging model
Logging CV results matrix
Logging extra data related to the experiment
file:///D:/Data_Science/mlflow/mlruns/1/bafd195ebe8a48b8a216a2e155b68692/artifacts
runID: 5421c970c3084c5a82b66f6449412993
Logging parameters
Logging metrics
Logging model
Logging CV results matrix
Logging extra data related to the experiment
file:///D:/Data_Science/mlflow/mlruns/1/061163ffac4644929df8185645111323/artifacts
runID: 2a392fb645e0492da347902ac8bb2071
Logging parameters
Logging m

In [8]:

for i in range(len(alpha_list)):
#     alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.05
#     l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.05
   
   # Run ElasticNet
    lr = ElasticNet(alpha=alpha_list[i], l1_ratio=la_ratio_list[i], random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
   
   # Print out ElasticNet model metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha_list[i], la_ratio_list[i]))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    # Log mlflow attributes for mlflow UI
    mlflow.log_param("alpha", alpha_list[i])
    mlflow.log_param("l1_ratio", la_ratio_list[i])
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.010000, l1_ratio=0.100000):
  RMSE: 70.90385970638309
  MAE: 59.63089172068812
  R2: 0.23986643454764633
Elasticnet model (alpha=0.020000, l1_ratio=0.200000):
  RMSE: 73.95145365073573
  MAE: 62.37399980882025
  R2: 0.17311790724233078


MlflowException: Changing param values is not allowed. Param with key='alpha' was already logged with value='0.01' for run ID='79531a6cf8c94ce7b4bfef4cf851b446'. Attempted logging new value '0.02'.

In [33]:
import sklearn
def log_run(gridsearch: sklearn.model_selection.RandomizedSearchCV, experiment_name: str, model_name: str, run_index: int, conda_env, tags={}):
    """Logging of cross validation results to mlflow tracking server
    
    Args:
        experiment_name (str): experiment name
        model_name (str): Name of the model
        run_index (int): Index of the run (in Gridsearch)
        conda_env (str): A dictionary that describes the conda environment (MLFlow Format)
        tags (dict): Dictionary of extra data and tags (usually features)
    """
    
    cv_results = gridsearch.cv_results_
    with mlflow.start_run(run_name=str(run_index)) as run:  

        mlflow.log_param("folds", gridsearch.cv)

        print("Logging parameters")
        params = list(gridsearch.param_distributions.keys())
        for param in params:
            mlflow.log_param(param, cv_results["param_%s" % param][run_index])

        print("Logging metrics")
        for score_name in [score for score in cv_results if "mean_test" in score]:
            mlflow.log_metric(score_name, cv_results[score_name][run_index])
            mlflow.log_metric(score_name.replace("mean","std"), cv_results[score_name.replace("mean","std")][run_index])

        print("Logging model")        
        mlflow.sklearn.log_model(gridsearch.best_estimator_, model_name, conda_env=conda_env)

        print("Logging CV results matrix")
        tempdir = r"D:\Data_Science\mlflow\mlruns\cv_results_matrix"
        try:
            os.mkdir(tempdir)
        except FileExistsError :
            pass
        timestamp = datetime.now().isoformat().split(".")[0].replace(":", ".")
        filename = "%s-%s-cv_results.csv" % (model_name, timestamp)
        csv = os.path.join(tempdir, filename)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            pd.DataFrame(cv_results).to_csv(csv, index=False)
        
        mlflow.log_artifact(csv, "cv_results") 

        print("Logging extra data related to the experiment")
        if not bool(tags):
            mlflow.set_tags(tags) 

        run_id = run.info.run_uuid
        experiment_id = run.info.experiment_id
        mlflow.end_run()
        print(mlflow.get_artifact_uri())
        print("runID: %s" % run_id)

def log_results(gridsearch: sklearn.model_selection.RandomizedSearchCV, experiment_name, model_name, tags={}, log_only_best=False):
    """Logging of cross validation results to mlflow tracking server
    
    Args:
        experiment_name (str): experiment name
        model_name (str): Name of the model
        tags (dict): Dictionary of extra tags
        log_only_best (bool): Whether to log only the best model in the gridsearch or all the other models as well
    """
    conda_env = {
            'name': 'mlflow-env',
            'channels': ['defaults'],
            'dependencies': [
                'python=3.7.0',
                'scikit-learn>=0.21.3',
                {'pip': ['xgboost==1.0.1']}
            ]
        }


    best = gridsearch.best_index_

    mlflow.set_tracking_uri("")
    mlflow.set_experiment(experiment_name)

    if(log_only_best):
        log_run(gridsearch, experiment_name, model_name, best, conda_env, tags)
    else:
        for i in range(len(gridsearch.cv_results_['params'])):
            log_run(gridsearch, experiment_name, model_name, i, conda_env, tags)


In [19]:
mlflow.get_tracking_uri()

'http://kubernetes.docker.internal:5000'

In [7]:
alpha_list=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]
la_ratio_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in range(len(alpha_list)):
#     alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.05
#     l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.05
   
   # Run ElasticNet
    lr = ElasticNet(alpha=alpha_list[i], l1_ratio=la_ratio_list[i], random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
   
   # Print out ElasticNet model metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha_list[i], la_ratio_list[i]))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    # Log mlflow attributes for mlflow UI
    with mlflow.start_run(nested=True) as mlrun:
        mlflow.log_param("alpha", alpha_list[i])
        mlflow.log_param("l1_ratio", la_ratio_list[i])
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.010000, l1_ratio=0.100000):
  RMSE: 70.90385970638309
  MAE: 59.63089172068812
  R2: 0.23986643454764633
Elasticnet model (alpha=0.020000, l1_ratio=0.200000):
  RMSE: 73.95145365073573
  MAE: 62.37399980882025
  R2: 0.17311790724233078
Elasticnet model (alpha=0.030000, l1_ratio=0.300000):
  RMSE: 75.32029028937819
  MAE: 63.55824306139664
  R2: 0.1422235343526307
Elasticnet model (alpha=0.040000, l1_ratio=0.400000):
  RMSE: 75.96682341423187
  MAE: 64.09951695478446
  R2: 0.127434395913956
Elasticnet model (alpha=0.050000, l1_ratio=0.500000):
  RMSE: 76.18301406628095
  MAE: 64.27692872016821
  R2: 0.12246093670334146
Elasticnet model (alpha=0.060000, l1_ratio=0.600000):
  RMSE: 76.04147045998621
  MAE: 64.15861977716676
  R2: 0.12571874049827847
Elasticnet model (alpha=0.070000, l1_ratio=0.700000):
  RMSE: 75.48077490256487
  MAE: 63.688145183703945
  R2: 0.13856431903468613
Elasticnet model (alpha=0.080000, l1_ratio=0.800000):
  RMSE: 74.2341239253442
  MAE:

In [ ]:
alpha_list=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]
la_ratio_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in range(len(alpha_list)):
#     alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.05
#     l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.05
   
   # Run ElasticNet
    lr = ElasticNet(alpha=alpha_list[i], l1_ratio=la_ratio_list[i], random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
   
   # Print out ElasticNet model metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha_list[i], la_ratio_list[i]))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    # Log mlflow attributes for mlflow UI
    with mlflow.start_run(nested=True) as mlrun:
        mlflow.log_param("alpha", alpha_list[i])
        mlflow.log_param("l1_ratio", la_ratio_list[i])
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(lr, "model")

In [ ]:
# Compute paths
eps = 5e-3  # the smaller it is the longer is the path

print("Computing regularization path using the elastic net.")
alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=False)

# Display results
fig = plt.figure(1)
ax = plt.gca()

colors = cycle(['b', 'r', 'g', 'c', 'k'])
neg_log_alphas_enet = -np.log10(alphas_enet)
for coef_e, c in zip(coefs_enet, colors):
    l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle='--', c=c)

plt.xlabel('-Log(alpha)')
plt.ylabel('coefficients')
title = 'ElasticNet Path by alpha for l1_ratio = ' + str(l1_ratio)
plt.title(title)
plt.axis('tight')


# Save figures
fig.savefig("ElasticNet-paths.png")

# Close plot
plt.close(fig)

# Log artifacts (output files)
mlflow.log_artifact("ElasticNet-paths.png")